<a href="https://colab.research.google.com/github/woosterheert/rsvqa/blob/main/notebooks/prithvi_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🛰 **Remote Sensing Visual Question Answering**



1. We pull all the necesary code from our repository:

In [1]:
!git clone https://github.com/woosterheert/rsvqa

Cloning into 'rsvqa'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 46 (delta 14), reused 25 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (46/46), 30.14 KiB | 15.07 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [ ]:
!cd /content/rsvqa
!git pull

fatal: not a git repository (or any of the parent directories): .git


2. We connect our notebook to our google cloud storage bucket:

In [2]:
cd /content/rsvqa

/content/rsvqa


In [3]:
from google.colab import auth
auth.authenticate_user()

!echo "deb https://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt -qq update && apt -qq install gcsfuse

mount_path = "ou-genai-data"
local_path = f"/mnt/{mount_path}"

!mkdir -p {local_path}
!gcsfuse --implicit-dirs {mount_path} {local_path}

deb https://packages.cloud.google.com/apt gcsfuse-jammy main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1022  100  1022    0     0  14050      0 --:--:-- --:--:-- --:--:-- 14194
OK
53 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: https://packages.cloud.google.com/apt/dists/gcsfuse-jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 53 not upgraded.
Need to get 14.6 MB of archives.
After this operation, 0 B of additional disk space will be used.
Selecting previously unsele

3. We install necessary packages

In [4]:
!pip install rasterio
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 58.3 MB/s eta 0:00:00


In [9]:
from external.prithvi_mae import PrithviViT
from utils.data_utils import RSVQADataset
from transformers import BertModel, BertTokenizer
from torch.utils.data import DataLoader
import pandas as pd
import yaml
from models.dual_encoder import dual_encoder_with_classifier
import torch
import pytorch_lightning as pl
from torchvision.models import resnet50

4. Load all the necessary data

In [7]:
weights_path = "/mnt/ou-genai-data/Prithvi_EO_V1_100M.pt"
model_cfg_path = "/mnt/ou-genai-data/prithvi_config.yaml"
with open(model_cfg_path) as f:
    model_config = yaml.safe_load(f)

model_args, train_args = model_config["model_args"], model_config["train_params"]
model_args["num_frames"] = 1
model_args["encoder_only"] = True

In [8]:
df = pd.read_csv('/mnt/ou-genai-data/questions_and_answers_binary.csv', index_col=0)

In [10]:
df_train = df.query("split == 'train'")[:100]
df_val = df.query("split == 'validation'")[:100]
df_test = df.query("split == 'test'")[:100]

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [13]:
train_ds = RSVQADataset(df_train, train_args, '/mnt/ou-genai-data/6d_data', tokenizer)
val_ds = RSVQADataset(df_val, train_args, '/mnt/ou-genai-data/6d_data', tokenizer)
test_ds = RSVQADataset(df_test, train_args, '/mnt/ou-genai-data/6d_data', tokenizer)

In [14]:
train_dataloader = DataLoader(train_ds, batch_size=8, num_workers=4)
val_dataloader = DataLoader(val_ds, batch_size=8, num_workers=4)
test_dataloader = DataLoader(test_ds, batch_size=8, num_workers=4)

5. Create the encoders

In [16]:
#Pritvhi vision encoder:
checkpoint = torch.load(weights_path, map_location="cpu")
prithvi_encoder = PrithviViT(**model_args)
del checkpoint['encoder.pos_embed']
del checkpoint['decoder.decoder_pos_embed']
_ = prithvi_encoder.load_state_dict(checkpoint, strict=False)

<ipython-input-16-767ea7d31dda>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(weights_path, map_location="cpu")


In [17]:
resnet_encoder = resnet50(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 107MB/s]


In [18]:
bert_encoder = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

6. Create & train the models


6a. Resnet with BERT on RGB data

In [21]:
train_ds = RSVQADataset(df_train, train_args, '/mnt/ou-genai-data/rgb_data', tokenizer, add_temporal_dimension=False)
val_ds = RSVQADataset(df_val, train_args, '/mnt/ou-genai-data/rgb_data', tokenizer, add_temporal_dimension=False)
test_ds = RSVQADataset(df_test, train_args, '/mnt/ou-genai-data/rgb_data', tokenizer, add_temporal_dimension=False)

In [19]:
baseline_model = dual_encoder_with_classifier(resnet_encoder, bert_encoder, 1000, 768, use_vision_cls_token=False)
baseline_model_with_prithvi = dual_encoder_with_classifier(prithvi_encoder, bert_encoder, 768, 768, use_vision_cls_token=True)

In [20]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=baseline_model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:You are using a CUDA device ('NVIDIA L4') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                 | Type       | Params | Mode 
------------------------------------------------------------
0 | vision_encoder       | ResNet     | 25.6 M | train
1 | text_encoder         | BertModel  | 109 M  | eval 
2 | f

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Expected 3D (unbatched) or 4D (batched) input to conv2d, but got input of size: [8, 6, 1, 224, 224]

In [ ]:
trainer = pl.Trainer(max_epochs=10)
trainer.fit(model=baseline_model_with_prithvi, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)